In [3]:
import json
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pandas import json_normalize
from datetime import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from cfuzzyset import cFuzzySet as FuzzySet

In [5]:
def clean_text_2 (row):
    no_punc = re.sub(r'[^\w\s]', '', row)
    lower = no_punc.lower()
    no_space = re.sub(r'\s+', '', lower)
    return no_space

def extract_investment_rows (df, start_cell_str, end_cell_str):
    try:
        start_row = df[df.iloc[:, 0].apply(clean_text_2) == start_cell_str].index[0]
        end_row = df[df.iloc[:, 0].apply(clean_text_2) == end_cell_str].index[0]
    except IndexError:
        print("Start or end cell string not found in the DataFrame.")
        return
    portion = df.iloc[start_row + 1 : end_row + 1, :]
    return portion

def top_score (row, terms):
    scores = []
    standard_row = clean_text_2 (row)
    for search_string in terms:
        standard_term = clean_text_2 (search_string)
        score = fuzz.partial_token_set_ratio (standard_row, standard_term)
        # if standard_row in standard_term:
        #     score += 10
        scores.append(score)
    if score >= 45:
        return max(scores)
    else: return 0

def best_match (string, dict):
    best_match, best_match_type, best_match_score = '', '', 0
    for key in dict:
        score = top_score (string, key['terms'])
        if score > best_match_score:
            best_match, best_match_type, best_match_score = key['category'], key['type'], score
    if best_match_score <= 60:
        best_match, best_match_type = 'No match', 'No match'
    return best_match, best_match_type, best_match_score

def find_income (row):
    if 'income' in row:
        return row.split('income')[0]
    else: return row

In [14]:
def generate_types (df):
    new_df = df.copy()
    headings = []
    subheadings = []

    current_heading, current_heading_type = 'No match', 'investment'
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
           heading_guess, heading_guess_type, _ = best_match (row[0], investment_categories + income_categories)
           # heading_guess, heading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           current_heading, current_heading_type = heading_guess, heading_guess_type
        headings.append((current_heading, current_heading_type))
    new_df['headings'] = headings

    for i in range(len(df)):
        row = df.iloc[i]
        _, current_heading_type = row['headings']
        if not(pd.isnull(row[1]) and pd.isnull(row[2])):
           if current_heading_type == 'investment':
               subheading_guess, subheading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           if current_heading_type == 'income':
                subheading_guess, subheading_guess_type, _ = best_match (row[0], investment_categories)
           subheadings.append((subheading_guess, subheading_guess_type))
        else:
            subheadings.append(('No match', 'investment'))
    new_df['subheadings'] = subheadings
    
    return new_df

In [64]:
def generate_headings (df):
    new_df = df.copy()
    headings = []
    current_heading, current_heading_type = 'No match', 'investment'
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
           heading_guess, heading_guess_type, _ = best_match (row[0], investment_categories + income_categories)
           # heading_guess, heading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           current_heading, current_heading_type = heading_guess, heading_guess_type
        headings.append((current_heading, current_heading_type))
    # Account for last row
    headings[-1] = ('total', 'investment')
    new_df['headings'] = headings
    return new_df

In [65]:
def generate_subheadings (df):
    new_df = df.copy()
    subheadings = []
    for i in range(len(df)):
        row = df.iloc[i]
        _, current_heading_type = row['headings']
        if not(pd.isnull(row[1]) and pd.isnull(row[2])):
           if current_heading_type == 'investment':
               subheading_guess, subheading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           if current_heading_type == 'income':
                subheading_guess, subheading_guess_type, _ = best_match (row[0], investment_categories)
           subheadings.append((subheading_guess, subheading_guess_type))
        else:
            subheadings.append(('No match', 'investment'))
    # Account for last row
    subheadings[-1] = ('total', 'income')
    new_df['subheadings'] = subheadings
    return new_df

def generate_coordinates (df):
    return generate_subheadings(generate_headings(df))

In [13]:
investment_categories = [
    {
        'category': 'non_affiliated',
        'type': 'investment',
        'terms': ['noncontrolledunaffiliatedinvestments', 
                  'noncontrollednonaffiliatecompanyinvestments',
                  'noncontrollednonaffiliatedinvestments',
                  'noncontrolnonaffiliateinvestments']
    },
    {
        'category': 'affiliated',
        'type': 'investment',
        'terms': ['noncontrolledaffiliatedinvestments', 
                  'noncontrolledaffiliatecompanyinvestments',
                  'noncontrolledaffiliatedinvestments',
                  'affiliateinvestments']
    },
    {
        'category': 'control',
        'type': 'investment',
        'terms': ['controlledaffiliatedinvestments',
                  'controlledaffiliatecompanyinvestments',
                  'controlledaffiliatedinvestments',
                  'controlinvestments']
    },
    ]

income_categories = [
    {
        'category': 'interest',
        'type': 'income',
        'terms': ['interest',
                  'incomeexcluding']
    },
    {
        'category': 'total',
        'type': 'income',
        'terms': ['total', 
                  'interestfeeanddividend',
                  'totalinvestment']
    },
    {
        'category': 'dividend',
        'type': 'income',
        'terms': ['dividend']
    },
    { 
        'category': 'pik',
        'type': 'income',
        'terms': ['pik', 'paymentinkind', 'paidinkind']
    },
    {
        'category': 'fee',
        'type': 'income',
        'terms': ['fee']
    },
    {
        'category': 'other',
        'type': 'income',
        'terms': ['other']
    }
]

In [67]:
def create_investment_dicts ():
    investment_dicts = []
    for investment in ['non_affiliated', 'affiliated', 'control']:
        for income in ['interest', 'total', 'dividend', 'pik', 'fee', 'other']:
            investment_dicts.append({
                'investment': investment,
                'income': income,
                'value1': pd.NA,
                'value2': pd.NA
            })
    investment_dicts.append({
        'investment': 'total',
        'income': 'total',
        'value1': pd.NA,
        'value2': pd.NA
    })
    return investment_dicts

In [71]:
def format_in_dicts (df):
    new_dicts = create_investment_dicts ()
    for i in range(len(df)):
        row = df.iloc[i]
        heading, _ = row['headings']
        subheading, _ = row['subheadings']
        for dict in new_dicts:
            if ((dict['investment'] == heading and dict['income'] == subheading) 
                or (dict['investment'] == subheading and dict['income'] == heading)):
                dict['value1'] = row[1]
                dict['value2'] = row[2]
    return new_dicts

In [72]:
df = pd.read_csv('ares_operations.csv')

In [73]:
df_investments = extract_investment_rows(df, 'investmentincome', 'totalinvestmentincome')
df_investments

,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022
1,From non-controlled/non-affiliate company inve...,NaN,NaN
2,Interest income (excluding payment-in-kind (“P...,367000000.0,242000000.0
3,PIK interest income,37000000.0,27000000.0
4,Capital structuring service fees,10000000.0,26000000.0
5,Dividend income,57000000.0,41000000.0
6,Other income,16000000.0,10000000.0
7,Total investment income from non-controlled/no...,487000000.0,346000000.0
8,From non-controlled affiliate company investme...,NaN,NaN
9,Interest income (excluding PIK interest income),3000000.0,1000000.0
10,PIK interest income,1000000.0,1000000.0


In [74]:
formatted_df = generate_coordinates(df_investments)
formatted_df


,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022,headings,subheadings
1,From non-controlled/non-affiliate company inve...,NaN,NaN,"(non_affiliated, investment)","(No match, investment)"
2,Interest income (excluding payment-in-kind (“P...,367000000.0,242000000.0,"(non_affiliated, investment)","(interest, income)"
3,PIK interest income,37000000.0,27000000.0,"(non_affiliated, investment)","(pik, income)"
4,Capital structuring service fees,10000000.0,26000000.0,"(non_affiliated, investment)","(fee, income)"
5,Dividend income,57000000.0,41000000.0,"(non_affiliated, investment)","(dividend, income)"
6,Other income,16000000.0,10000000.0,"(non_affiliated, investment)","(other, income)"
7,Total investment income from non-controlled/no...,487000000.0,346000000.0,"(non_affiliated, investment)","(total, income)"
8,From non-controlled affiliate company investme...,NaN,NaN,"(affiliated, investment)","(No match, investment)"
9,Interest income (excluding PIK interest income),3000000.0,1000000.0,"(affiliated, investment)","(interest, income)"
10,PIK interest income,1000000.0,1000000.0,"(affiliated, investment)","(pik, income)"


In [75]:
format_in_dicts(formatted_df)

[{'investment': 'non_affiliated',
  'income': 'interest',
  'value1': '367000000.0',
  'value2': 242000000.0},
 {'investment': 'non_affiliated',
  'income': 'total',
  'value1': '487000000.0',
  'value2': 346000000.0},
 {'investment': 'non_affiliated',
  'income': 'dividend',
  'value1': '57000000.0',
  'value2': 41000000.0},
 {'investment': 'non_affiliated',
  'income': 'pik',
  'value1': '37000000.0',
  'value2': 27000000.0},
 {'investment': 'non_affiliated',
  'income': 'fee',
  'value1': '10000000.0',
  'value2': 26000000.0},
 {'investment': 'non_affiliated',
  'income': 'other',
  'value1': '16000000.0',
  'value2': 10000000.0},
 {'investment': 'affiliated',
  'income': 'interest',
  'value1': '3000000.0',
  'value2': 1000000.0},
 {'investment': 'affiliated',
  'income': 'total',
  'value1': '4000000.0',
  'value2': 2000000.0},
 {'investment': 'affiliated',
  'income': 'dividend',
  'value1': <NA>,
  'value2': <NA>},
 {'investment': 'affiliated',
  'income': 'pik',
  'value1': '10